In [1]:
using OrdinaryDiffEq
using LinearAlgebra
using Pkg
using QuantumOpticsBase

In [9]:
Pkg.add("QuantumOpticsBase")

   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`


  [4f57444f] + QuantumOpticsBase v0.4.18
    Updating `~/.julia/environments/v1.9/Manifest.toml`


  [4f57444f] ~ QuantumOpticsBase v0.4.18 `https://github.com/Krastanov/QuantumOpticsBase.jl.git#master` ⇒ v0.4.18


In [10]:
Pkg.add(url="https://github.com/Krastanov/QuantumOpticsBase.jl.git")

    Updating git-repo `https://github.com/Krastanov/QuantumOpticsBase.jl.git`


   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`


  [4f57444f] + QuantumOpticsBase v0.4.18 `https://github.com/Krastanov/QuantumOpticsBase.jl.git#master`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
b_fock = FockBasis(50);
op = dm(coherentstate(b_fock, 1.0+0.0im));

In [3]:
bf = FockBasis(3)
op3 = randoperator(bf)

Operator(dim=4x4)
  basis: Fock(cutoff=3)
 0.688142+0.309946im  0.392646+0.612445im  …  0.989957+0.931753im
 0.633537+0.227422im  0.877231+0.472045im     0.786938+0.331903im
 0.597537+0.189748im  0.121084+0.500887im     0.245598+0.346756im
 0.796544+0.70605im   0.404363+0.232892im     0.339475+0.822821im

In [6]:
@inline Base.axes(A::DataOperator) = tuple(findall(!isnothing, A.data))
Base.getindex(op::DataOperator, inds::Vararg{Int,2}) = op.data[inds...]
Base.setindex!(op::DataOperator, val, inds::Vararg{Int,2}) = op.data[inds...] = val

function Broadcast.BroadcastStyle(op::T, a::Broadcast.DefaultArrayStyle{0}) where {Bl<:Basis, Br<:Basis, T<:QuantumOpticsBase.OperatorStyle{Bl,Br}}
    return T()
end

In [4]:
#Base.:+(x::DataOperator,y::my_type) = my_type(x.data+y.data,x.name)
Base.:*(x::Number,y::DataOperator{BL, BR}) where {BL, BR} = Operator{BL,BR}(y.basis_l, y.basis_r, x*y.data)
Base.:/(x::DataOperator{BL, BR},y::Number) where {BL, BR} = Operator{BL,BR}(y.basis_l, y.basis_r, x.data / y)

In [10]:
Base.Broadcast.extrude(x::DataOperator) = x

In [11]:
op .+ (2.0+0.0im)

MethodError: MethodError: no method matching find_dType(::Base.Broadcast.Broadcasted{OperatorStyle{FockBasis{Int64}, FockBasis{Int64}}, Tuple{Vector{CartesianIndex{2}}}, typeof(+), Tuple{Operator{FockBasis{Int64}, FockBasis{Int64}, Matrix{ComplexF64}}, ComplexF64}})

Closest candidates are:
  find_dType(!Matched::DataOperator, !Matched::Any)
   @ Main ~/Documents/EPFL/PhD/Projects/CS-TDVP/TDVP/Optimized_CS_TDVP/examples/qo_diffeqtype.ipynb:32


In [3]:
function rhs_test2_oop(du, u, p, t)
    du .= u
end

rhs_test2_oop (generic function with 1 method)

In [57]:
using Lazy
using RecursiveArrayTools

@forward DataOperator.data (Base.length, Base.iterate, Base.getindex, Base.setindex!, Base.size)

Base.similar(u::Operator{BL, BR}, type::Type{L}=eltype(u)) where {BL, BR, L} =  Operator{BL,BR}(u.basis_l, u.basis_r, similar(u.data, type))
RecursiveArrayTools.recursivefill!(op::Operator{BL, BR}, a) where {BL, BR} =  recursivefill!(op.data, a)

function Base.similar(bc::Base.Broadcast.Broadcasted{QuantumOpticsBase.OperatorStyle{BL, BR}}, ::Type{ElType}) where {BL, BR, ElType}
    bl,br = QuantumOptics.find_basis(bcf.args)
    bc_ = Broadcasted_restrict_f(bcf.f, bcf.args, axes(bcf))
    Operator{BL, BR}(bc_.basis_l, bc_.basis_r, similar(Matrix{ElType}, axes(bc)))
end

QuantumOpticsBase.OperatorStyle(::Val{BL},::Val{BR}) where {BL, BR} = QuantumOpticsBase.OperatorStyle{BL, BR}()
QuantumOpticsBase.OperatorStyle{M}(::Val{BL},::Val{BR}) where {M, BL, BR} = QuantumOpticsBase.OperatorStyle{BL, BR}()

In [6]:
"""From QuantumOpticsBase, pull request https://github.com/qojulia/QuantumOpticsBase.jl/pull/16"""
# Broadcasting
Base.size(A::DataOperator) = size(A.data)
Base.size(A::DataOperator, d::Int) = size(A.data, d)
@inline Base.axes(A::DataOperator) = tuple(findall(!isnothing, A.data))
Base.broadcastable(A::DataOperator) = A

# Custom broadcasting styles
abstract type DataOperatorStyle{BL<:Basis,BR<:Basis} <: Broadcast.BroadcastStyle end
struct OperatorStyle{BL<:Basis,BR<:Basis} <: DataOperatorStyle{BL,BR} end

# Style precedence rules
Broadcast.BroadcastStyle(::Type{<:Operator{BL,BR}}) where {BL<:Basis,BR<:Basis} = OperatorStyle{BL,BR}()
Broadcast.BroadcastStyle(::OperatorStyle{B1,B2}, ::OperatorStyle{B3,B4}) where {B1<:Basis,B2<:Basis,B3<:Basis,B4<:Basis} = throw(IncompatibleBases())

# Broadcast with scalars (of use in ODE solvers checking for tolerances, e.g. `.* reltol .+ abstol`)
Broadcast.BroadcastStyle(::T, ::Broadcast.DefaultArrayStyle{0}) where {Bl<:Basis, Br<:Basis, T<:OperatorStyle{Bl,Br}} = T()

# Out-of-place broadcasting
@inline function Base.copy(bc::Broadcast.Broadcasted{Style,Axes,F,Args}) where {BL<:Basis,BR<:Basis,Style<:OperatorStyle{BL,BR},Axes,F,Args<:Tuple}
    bcf = Broadcast.flatten(bc)
    bl,br = find_basis(bcf.args...)
    T = find_dType(bcf)
    data = zeros(T, length(bl), length(br))
    @inbounds @simd for I in eachindex(bcf)
        data[I] = bcf[I]
    end
    return Operator{BL,BR}(bl, br, data)
end

find_basis(a::DataOperator, rest) = (a.basis_l, a.basis_r)
find_dType(a::DataOperator, rest) = eltype(a)
Base.getindex(a::DataOperator, idx) = getindex(a.data, idx)
Base.iterate(a::DataOperator) = iterate(a.data)
Base.iterate(a::DataOperator, idx) = iterate(a.data, idx)

# In-place broadcasting
@inline function Base.copyto!(dest::DataOperator{BL,BR}, bc::Broadcast.Broadcasted{Style,Axes,F,Args}) where {BL<:Basis,BR<:Basis,Style<:DataOperatorStyle{BL,BR},Axes,F,Args}
    axes(dest) == axes(bc) || Base.Broadcast.throwdm(axes(dest), axes(bc))
    bc′ = Base.Broadcast.preprocess(dest, bc)
    dest′ = dest.data
    @inbounds @simd for I in eachindex(bc′)
        dest′[I] = bc′[I]
    end
    return dest
end
@inline Base.copyto!(A::DataOperator{BL,BR},B::DataOperator{BL,BR}) where {BL<:Basis,BR<:Basis} = (copyto!(A.data,B.data); A)
@inline Base.copyto!(dest::DataOperator{BL,BR}, bc::Broadcast.Broadcasted{Style,Axes,F,Args}) where {BL<:Basis,BR<:Basis,Style<:DataOperatorStyle,Axes,F,Args} =
    throw(IncompatibleBases())

# A few more standard interfaces: These do not necessarily make sense for a StateVector, but enable transparent use of DifferentialEquations.jl
Base.eltype(::Type{Operator{Bl,Br,A}}) where {Bl,Br,N,A<:AbstractMatrix{N}} = N # ODE init
Base.any(f::Function, ρ::Operator; kwargs...) = any(f, ρ.data; kwargs...) # ODE nan checks
Base.all(f::Function, ρ::Operator; kwargs...) = all(f, ρ.data; kwargs...)
Broadcast.similar(ρ::Operator, t) = typeof(ρ)(ρ.basis_l, ρ.basis_r, copy(ρ.data))
using RecursiveArrayTools
RecursiveArrayTools.recursivecopy!(dst::Operator{Bl,Br,A},src::Operator{Bl,Br,A}) where {Bl,Br,A} = copy!(dst.data,src.data) # ODE in-place equations

In [5]:
similar(op);
recursivefill!(op, 1.0);

MethodError: MethodError: no method matching similar(::Operator{FockBasis{Int64}, FockBasis{Int64}, Matrix{ComplexF64}})

Closest candidates are:
  similar(::Operator, !Matched::Any)
   @ Main ~/Documents/EPFL/PhD/Projects/CS-TDVP/TDVP/Optimized_CS_TDVP/examples/qo_diffeqtype.ipynb:55
  similar(!Matched::Union{Adjoint{T, var"#s970"}, Transpose{T, var"#s970"}} where {T, var"#s970"<:(AbstractVector)})
   @ LinearAlgebra ~/Programs/julia-1.9.0/share/julia/stdlib/v1.9/LinearAlgebra/src/adjtrans.jl:329
  similar(!Matched::Union{Adjoint{T, var"#s970"}, Transpose{T, var"#s970"}} where {T, var"#s970"<:(AbstractVector)}, !Matched::Type{T}) where T
   @ LinearAlgebra ~/Programs/julia-1.9.0/share/julia/stdlib/v1.9/LinearAlgebra/src/adjtrans.jl:330
  ...


In [61]:
#Check the broadcast style of op
Broadcast.BroadcastStyle(typeof(op))

OperatorStyle{FockBasis{Int64}, FockBasis{Int64}}()

In [25]:
op .* 2

MethodError: MethodError: no method matching find_dType(::Base.Broadcast.Broadcasted{OperatorStyle{FockBasis{Int64}, FockBasis{Int64}}, Tuple{Vector{CartesianIndex{2}}}, typeof(*), Tuple{Operator{FockBasis{Int64}, FockBasis{Int64}, Matrix{ComplexF64}}, Int64}})

Closest candidates are:
  find_dType(!Matched::DataOperator, !Matched::Any)
   @ Main ~/Documents/EPFL/PhD/Projects/CS-TDVP/TDVP/Optimized_CS_TDVP/examples/qo_diffeqtype.ipynb:32


In [52]:
prob = ODEProblem(rhs_test2_oop, op, (0.0, 1.0))
sol   = solve(prob, RK4())

ArgumentError: ArgumentError: Broadcasted{Unknown} wrappers do not have a style assigned